In [1]:
import os
import random

import numpy as np
import cv2
from glob import glob
from scipy.ndimage.interpolation import rotate
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
#Cac ham tang cuong anh duoc dinh nghia boi tac gia
def read_image(imagefile, grayscale=False):
    if grayscale == True:
        image = cv2.imread(imagefile)
        #image = np.expand_dims(image, -1)
    else:
        image = cv2.imread(imagefile)
    return image

def save_image(image, mask, path, binary=True):
    image = np.array(image)
    if binary == True:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path[0], image)
    cv2.imwrite(path[1], mask)

def concat_images(images, rows, cols):
    _, h, w, _ = images.shape
    images = images.reshape((rows, cols, h, w, 3))
    images = images.transpose(0, 2, 1, 3, 4)
    images = images.reshape((rows * h, cols * w, 3))
    return images

def check_size(size):
    if type(size) == int:
        size = (size, size)
    if type(size) != tuple:
        raise TypeError('size is int or tuple')
    return size

def subtract(image):
    image = image / 255
    return image

def resize(image, size):
    size = check_size(size)
    image = cv2.resize(image, size)
    return image

def center_crop(image, mask, crop_size, size):
    h, w, _ = image.shape
    crop_size = check_size(crop_size)
    top = (h - crop_size[0]) // 2
    left = (w - crop_size[1]) // 2
    bottom = top + crop_size[0]
    right = left + crop_size[1]

    image = image[top:bottom, left:right, :]
    mask = mask[top:bottom, left:right, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def random_crop(image, mask, crop_size, size):
    crop_size = check_size(crop_size)
    h, w, _ = image.shape
    top = np.random.randint(0, h - crop_size[0])
    left = np.random.randint(0, w - crop_size[1])
    bottom = top + crop_size[0]
    right = left + crop_size[1]

    image = image[top:bottom, left:right, :]
    mask = mask[top:bottom, left:right, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def horizontal_flip(image, mask, size):
    image = image[:, ::-1, :]
    mask = mask[:, ::-1, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def vertical_flip(image, mask, size):
    image = image[::-1, :, :]
    mask = mask[::-1, :, :]

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def scale_augmentation(image, mask, scale_range, crop_size, size):
    scale_size = np.random.randint(*scale_range)
    image = cv2.resize(image, (scale_size, scale_size))
    mask = cv2.resize(mask, (scale_size, scale_size))
    image, mask = random_crop(image, mask, crop_size, size)
    return image, mask

def random_rotation(image, mask, size, angle_range=(0, 90)):
    h1, w1, _ = image.shape
    h2, w2, _ = mask.shape

    angle = np.random.randint(*angle_range)
    image = rotate(image, angle)
    image = resize(image, (h1, w1))

    mask = rotate(mask, angle)
    mask = resize(mask, (h2, w2))

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def cutout(image_origin, mask_origin, mask_size, mask_value='mean'):
    image = np.copy(image_origin)
    mask = np.copy(mask_origin)

    if mask_value == 'mean':
        mask_value = image.mean()
    elif mask_value == 'random':
        mask_value = np.random.randint(0, 256)

    h, w, _ = image.shape
    top = np.random.randint(0 - mask_size // 2, h - mask_size)
    left = np.random.randint(0 - mask_size // 2, w - mask_size)
    bottom = top + mask_size
    right = left + mask_size
    if top < 0:
        top = 0
    if left < 0:
        left = 0

    image[top:bottom, left:right, :].fill(mask_value)
    mask[top:bottom, left:right, :].fill(0)

    image = resize(image, size)
    mask = resize(mask, size)

    return image, mask

def brightness_augment(img, mask, factor=0.5):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) #convert to hsv
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * (factor + np.random.uniform()) #scale channel V uniformly
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255 #reset out of range values
    rgb = cv2.cvtColor(np.array(hsv, dtype=np.uint8), cv2.COLOR_HSV2RGB)

    image = resize(rgb, size)
    mask = resize(mask, size)

    return image, mask

def rgb_to_grayscale(img, mask):
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = [img, img, img]
    img = np.transpose(img, (1, 2, 0))

    image = resize(img, size)
    mask = resize(mask, size)
    return image, mask

def create_dir(name):
    try:
        os.mkdir(name)
    except:
        pass



In [3]:
size = (256, 256)
crop_size = (300, 300)
full_path = ""

path = ""
dataset_name = ""
full_path = os.path.join(path, dataset_name)

new_full_path = "new_data_n/"

images = glob(os.path.join(full_path, "images/", "*"))
masks = glob(os.path.join(full_path, "masks/", "*"))

images.sort()
masks.sort()

len_ids = len(images)
train_size = int((80/100)*len_ids)
valid_size = int((10/100)*len_ids)		## Here 10 is the percent of images used for validation
test_size = int((10/100)*len_ids)		## Here 10 is the percent of images used for testing

train_images, valid_images = train_test_split(images, test_size=test_size, random_state=42)
train_masks, valid_masks = train_test_split(masks, test_size=test_size, random_state=42)

train_images, test_images = train_test_split(train_images, test_size=test_size, random_state=42)
train_masks, test_masks = train_test_split(train_masks, test_size=test_size, random_state=42)

print("Total Size: ", len_ids)
print("Training Size: ", train_size)
print("Validation Size: ", valid_size)
print("Testing Size: ", test_size)

    ## Validation images and masks
for idx, p in tqdm(enumerate(test_images), total=len(test_images)):
        ## Path
  name = p.split("/")[-1].split(".")[0]
  image_path = test_images[idx]
  mask_path = test_masks[idx]

  if os.path.exists(image_path) and os.path.exists(mask_path):
    image = read_image(image_path)
    mask = read_image(mask_path, grayscale=True)

    new_image_path = new_full_path + "test"+ "/images/"
    new_mask_path = new_full_path+ "test"+ "/masks/"

    image = resize(image, size)
    mask = resize(mask, size)

    img_path = new_image_path + str(name) + ".jpg"
    mask_path = new_mask_path + str(name) + ".jpg"
    tmp_path = [img_path, mask_path]
    save_image(image, mask, tmp_path)

    ## Testing images and masks
for idx, p in tqdm(enumerate(valid_images), total=len(valid_images)):
        ## Path
  name = p.split("/")[-1].split(".")[0]
  image_path = valid_images[idx]
  mask_path = valid_masks[idx]

  if os.path.exists(image_path) and os.path.exists(mask_path):
    image = read_image(image_path)
    mask = read_image(mask_path, grayscale=True)

    new_image_path = new_full_path + "valid"+ "/images/"
    new_mask_path = new_full_path+"valid"+ "/masks/"

    image = resize(image, size)
    mask = resize(mask, size)

    img_path = new_image_path + str(name) + ".jpg"
    mask_path = new_mask_path + str(name) + ".jpg"
    tmp_path = [img_path, mask_path]
    save_image(image, mask, tmp_path)

    ## Training images and masks
for idx, p in tqdm(enumerate(train_images), total=len(train_images)):
        ## Path
  name = p.split("/")[-1].split(".")[0]
  image_path = train_images[idx]
  mask_path = train_masks[idx]

  if os.path.exists(image_path) and os.path.exists(image_path):
    image = read_image(image_path)
    mask = read_image(mask_path, grayscale=True)

    ## Augment
    image3, mask3 = horizontal_flip(image, mask, size)
    image6, mask6 = random_rotation(image, mask, size)
    ## Extra Scale Augmentation
    image10, mask10 = scale_augmentation(image, mask, (540, 820), crop_size, size)
    ## Brightness
    image15, mask15 = brightness_augment(image, mask, factor=0.6)
  
    ## More Cutout
    image20, mask20 = random_crop(image, mask, crop_size, size)
    image21, mask21 = cutout(image, mask, 256)
    ## Grayscale
    image23, mask23 = rgb_to_grayscale(image, mask)

    ## Original image and mask
    image = resize(image, size)
    mask = resize(mask, size)
    ##Them
    
    ## All images and masks
    all_images = [image,image3,image6, 
                  image10, image15,image20,image21,
                image23]
    all_masks  = [mask,mask3,mask6, 
                  mask10, mask15,mask20,mask21,
                mask23]

    ## Save the images and masks
    new_image_path = new_full_path + "train"+ "/images/"
    new_mask_path = new_full_path+ "train"+ "/masks/"

    for j in range(len(all_images)):
      img_path = new_image_path + str(name) + "_" + str(j) + ".jpg"
      msk_path = new_mask_path + str(name) + "_" + str(j) + ".jpg"

      img = all_images[j]
      msk = all_masks[j]
      path = [img_path, msk_path]
      save_image(img, msk, path)

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100


100%|██████████| 800/800 [10:28<00:00,  1.27it/s]
